In [56]:
import numpy as np
import pandas as pd
import re
import statistics

## Read data from csv

In [3]:
data= pd.read_csv("/home/atharva/Documents/Project/Datasets/goodreadsbooks/books.csv", error_bad_lines=False)

b'Skipping line 4012: expected 10 fields, saw 11\nSkipping line 5688: expected 10 fields, saw 11\nSkipping line 7056: expected 10 fields, saw 11\nSkipping line 10601: expected 10 fields, saw 11\nSkipping line 10668: expected 10 fields, saw 11\n'


In [4]:
data.head()

,bookID,title,authors,average_rating,isbn,isbn13,language_code,# num_pages,ratings_count,text_reviews_count
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling-Mary GrandPré,4.56,0439785960,9780439785969,eng,652,1944099,26249
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling-Mary GrandPré,4.49,0439358078,9780439358071,eng,870,1996446,27613
2,3,Harry Potter and the Sorcerer's Stone (Harry P...,J.K. Rowling-Mary GrandPré,4.47,0439554934,9780439554930,eng,320,5629932,70390
3,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.41,0439554896,9780439554893,eng,352,6267,272
4,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling-Mary GrandPré,4.55,043965548X,9780439655484,eng,435,2149872,33964


## Check for null values

In [10]:
column_list=[]
for col in data.columns:
    column_list.append(col)
column_list

['bookID',
 'title',
 'authors',
 'average_rating',
 'isbn',
 'isbn13',
 'language_code',
 '# num_pages',
 'ratings_count',
 'text_reviews_count']

In [11]:
for columns in column_list:
    print(columns+": "+str(sum(data[columns].isna())))

bookID: 0
title: 0
authors: 0
average_rating: 0
isbn: 0
isbn13: 0
language_code: 0
# num_pages: 0
ratings_count: 0
text_reviews_count: 0


## There are no NA values present in our first dataset. 
## We check the next csv file now

In [122]:
data_2= pd.read_csv("/home/atharva/Documents/Project/Datasets/Book_Data/book_data.csv")

In [123]:
data_2.head()

,book_authors,book_desc,book_edition,book_format,book_isbn,book_pages,book_rating,book_rating_count,book_review_count,book_title,genres,image_url
0,Suzanne Collins,Winning will make you famous. Losing means cer...,NaN,Hardcover,9.78044E+12,374 pages,4.33,5519135,160706,The Hunger Games,Young Adult|Fiction|Science Fiction|Dystopia|F...,https://images.gr-assets.com/books/1447303603l...
1,J.K. Rowling|Mary GrandPré,There is a door at the end of a silent corrido...,US Edition,Paperback,9.78044E+12,870 pages,4.48,2041594,33264,Harry Potter and the Order of the Phoenix,Fantasy|Young Adult|Fiction,https://images.gr-assets.com/books/1255614970l...
2,Harper Lee,The unforgettable novel of a childhood in a sl...,50th Anniversary,Paperback,9.78006E+12,324 pages,4.27,3745197,79450,To Kill a Mockingbird,Classics|Fiction|Historical|Historical Fiction...,https://images.gr-assets.com/books/1361975680l...
3,Jane Austen|Anna Quindlen|Mrs. Oliphant|George...,«È cosa ormai risaputa che a uno scapolo in po...,"Modern Library Classics, USA / CAN",Paperback,9.78068E+12,279 pages,4.25,2453620,54322,Pride and Prejudice,Classics|Fiction|Romance,https://images.gr-assets.com/books/1320399351l...
4,Stephenie Meyer,About three things I was absolutely positive.F...,NaN,Paperback,9.78032E+12,498 pages,3.58,4281268,97991,Twilight,Young Adult|Fantasy|Romance|Paranormal|Vampire...,https://images.gr-assets.com/books/1361039443l...


### Columns like book_description, book_edition, book_format, image_url are currently not needed. So the best idea is to drop them

In [124]:
drop_columns= ['book_desc', 'book_edition', 'book_format', 'image_url']

In [125]:
for col in drop_columns:
    data_2= data_2.drop(columns=col)

In [126]:
data_2.head()

,book_authors,book_isbn,book_pages,book_rating,book_rating_count,book_review_count,book_title,genres
0,Suzanne Collins,9.78044E+12,374 pages,4.33,5519135,160706,The Hunger Games,Young Adult|Fiction|Science Fiction|Dystopia|F...
1,J.K. Rowling|Mary GrandPré,9.78044E+12,870 pages,4.48,2041594,33264,Harry Potter and the Order of the Phoenix,Fantasy|Young Adult|Fiction
2,Harper Lee,9.78006E+12,324 pages,4.27,3745197,79450,To Kill a Mockingbird,Classics|Fiction|Historical|Historical Fiction...
3,Jane Austen|Anna Quindlen|Mrs. Oliphant|George...,9.78068E+12,279 pages,4.25,2453620,54322,Pride and Prejudice,Classics|Fiction|Romance
4,Stephenie Meyer,9.78032E+12,498 pages,3.58,4281268,97991,Twilight,Young Adult|Fantasy|Romance|Paranormal|Vampire...


## Now we are interested in page count but currently it is in String format. Let's format it to display only number

In [127]:
print(sum(data_2.book_pages.isna()))
print(len(data_2.index))

2522
54301


### There are 2522 null values in this field and it's length is 54301

In [128]:
counter=0
page_list=[]
for ele in data_2.book_pages:
    if (counter< len(data_2.index)):
        if(type(ele)== type("")):
            pages, _=ele.split(" ")
            page_list.append(int(pages))
        

In [129]:
print(len(page_list))
print(statistics.mean(page_list))

51779
337.6628362849804


In [130]:
data_2['book_pages'].fillna(int(statistics.mean(page_list)), inplace=True)

In [131]:
print(sum(data_2.book_pages.isna()))
print(int(337.6628362849804))

0
337


In [132]:
pattern= re.compile(r'[\D]+')
test=data_2['book_pages'].values
for i in range(len(test)):
    if (type(test[i]) != int):
        test[i]= int(re.sub(pattern,'',test[i]))

In [133]:
counter=0
diff=0
for ele in data_2.book_pages:
    if ele!= test[counter]:
        diff+=1
    counter+=1
diff

0

In [134]:
data_2['book_page_count']= test

In [137]:
data_2= data_2.drop(columns='book_pages')